In [77]:
import numpy as np
import csv
import pickle
import pandas as pd

In [78]:
with open('../nTFs.txt', 'r') as f:
    nTFs = f.read().splitlines()

In [79]:
data_all = np.loadtxt('../data_complete.txt')
all_bin = np.vsplit(data_all, 6)
all_init = np.vstack([all_bin[i] for i in range(5)])
all_diff = np.vstack([all_bin[i+1]-all_bin[i] for i in range(5)])

def make_quad(X):
    quad = np.zeros((int(X.shape[0]), int(X.shape[1] + (X.shape[1]*(X.shape[1]-1))/2)))
    quad[:, :X.shape[1]] = np.copy(X)
    col = X.shape[1]
    for i in range(X.shape[1]-1):
        for j in range(i+1, X.shape[1]):
            quad[:,col] = (X[:,i]*X[:,j])
            col += 1
    return quad

quad = make_quad(all_init)
ranges = np.abs(np.max(quad, axis=0) - np.min(quad, axis=0))
medians = np.median(quad, axis=0)

In [80]:
w = np.zeros((4950,99))
bias = np.zeros(99)
for k in range(990):
    i = int(k/10)
    with open('./pickles/res/res_%s.pkl' % k, 'rb') as f:
        res = pickle.load(f)
    w[:,i:(i+1)] += res[0]
    bias[i] += res[1]
w = w/10

effects = np.multiply(np.abs(w), ranges[:,None])
np.fill_diagonal(effects, 0)
effects_sum = np.sum(effects, axis=1)
sorted_ind = np.argsort(effects_sum[:99])

In [81]:
with open('../nTFs.txt', 'r') as f:
    nTFs = f.read().splitlines()
with open('./pickles/names.pkl', 'rb') as f:
    names = pickle.load(f)

nTFs_list = []
for i in sorted_ind:
    if names[i] in nTFs:
        nTFs_list.append('1')
    else:
        nTFs_list.append('0')
        
nodes = pd.DataFrame(columns=['id', 'label', 'effect'])

for (i,ind) in enumerate(sorted_ind):
    nodes.loc[i] = [str(i), names[ind], effects_sum[ind]]
nodes['nTFs'] = nTFs_list
nodes.to_csv('./gephi/nodes_lin.csv', sep=',', index=False)

In [95]:
effects = np.multiply(np.abs(w), ranges[:,None])
np.fill_diagonal(effects, 0)
floor = np.min([np.max(effects[i,:]) for i in range(99)])

# edges = pd.DataFrame(columns=['Source', 'Target', 'Weight'])
# ind = 0
# for i in range(99):
#     for j in range(99):
#         if i != j:
#             edges.loc[ind] = [str(i), str(j), w[i,j]]
#             ind += 1
# edges.head()
# edges.to_csv('./gephi/edges_lin.csv', sep=',', index=False)

edges = pd.DataFrame(columns=['Source', 'Target', 'Weight', 'W', 'sign'])

ind = 0
for i in names:
    g1 = nodes.id[nodes.label == i].index[0]
#     index = np.argsort(effects[g1,:])[-5:]
    for j in names:
        if i != j:
            g2 = nodes.id[nodes.label == j].index[0]
            if effects[g1,g2] > floor and w[g1,g2] < 0:
                edges.loc[ind] = [str(g1), str(g2), np.abs(w[g1,g2]), w[g1,g2], str(0)]
                ind += 1
            if effects[g1,g2] >  floor and w[g1,g2] > 0:
                edges.loc[ind] = [str(g1), str(g2), np.abs(w[g1,g2]), w[g1,g2], str(1)]
                ind += 1
edges = edges.sort_values(by='Weight')
edges.to_csv('./gephi/edges_lin.csv', sep=',', index=False)
edges.head()

,Source,Target,Weight,W,sign
567,15,83,0.122170,0.122170,1
568,15,3,0.130308,0.130308,1
570,15,12,0.134788,0.134788,1
563,15,50,0.138196,-0.138196,0
565,15,88,0.138280,0.138280,1


In [6]:
effects_sum = np.sum(effects, axis=1)

quad_gene = []
for i in range(98):
    for j in range(i+1,99):
        quad_gene.append([i,j])
quad_pair = np.array(np.argsort(effects_sum[99:])[-10:])

nodes = pd.DataFrame(columns=['id', 'label'])

for (e,i) in enumerate(quad_pair):
    ind = 0
    nodes.loc[ind] = [str(i+99), str(names[quad_gene[i][0]] + '+' + names[quad_gene[i][1]])]
    for j in range(99):
        nodes.loc[j+1] = [j,names[j]]
    nodes.to_csv('./gephi/node_quad_%s.csv' % e, sep=',', index=False)

In [7]:
for (e,i) in enumerate(quad_pair):
    edges = pd.DataFrame(columns=['Source', 'Target', 'Weight', 'W', 'sign'])
    for j in range(99):
        if w[i+99,j] < 0:
            edges.loc[j] = [str(i+99), str(j), np.abs(w[i+99,j]), w[i+99,j], str(0)]
        else:
            edges.loc[j] = [str(i+99), str(j), np.abs(w[i+99,j]), w[i+99,j], str(1)]
    edges.to_csv('./gephi/edge_quad_%s.csv' % e, sep=',', index=False)